## Acquire Data

Mount the google drive which helps the following:

- `os.chdir('/content/drive/MyDrive/stock_charts_images')`
- `from utils import CustomViT as myViT`

In [ ]:
import os
import pickle

import numpy as np

In [ ]:
os.chdir('/content/drive/MyDrive/stock_charts_images')

In [ ]:
list_of_stock_chart_pkl_files = os.listdir()

In [ ]:
%%time

with open(list_of_stock_chart_pkl_files[3], 'rb') as f:
    data = pickle.load(f)

CPU times: user 1.53 ms, sys: 262 ms, total: 264 ms
Wall time: 2.11 s


In [ ]:
type(data), len(data)

(tuple, 2)

In [ ]:
type(data[0]), data[0].shape

(numpy.ndarray, (1089, 299, 299, 3))

In [ ]:
type(data[1]), data[1].shape

(numpy.ndarray, (1089,))

In [ ]:
X, y = data[0], data[1]

In [ ]:
X.min(), X.max()

(0, 255)

In [ ]:
X = X / 255

In [ ]:
X.min(), X.max()

(0.0, 1.0)

In [ ]:
y_binary = np.where(y >= 0.01, 1, 0)
y_binary.mean()

0.5133149678604224

## Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y_binary, test_size=0.2, random_state=42)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((871, 299, 299, 3), (218, 299, 299, 3), (871,), (218,))

## Define VIT

To run the code in a Jupyter notebook, you would first instantiate the `CustomViT` class, then use its methods to create the Vision Transformer (ViT) model, and finally train the model using the provided dataset. Below is a step-by-step guide on how to do this, assuming that the `CustomViT` class object is already defined in your notebook and you have your dataset loaded into `x_train`, `y_train`, `x_test`, and `y_test` variables.

1. **Instantiate the CustomViT Class**

First, create an instance of the `CustomViT` class. You can adjust the parameters based on your dataset and training preferences, or use the defaults as specified in the class definition.

```python
vit_model = CustomViT()
```

2. **Create the ViT Classifier Model**

Next, use the `create_vit_classifier` method to create the model.

```python
model = vit_model.create_vit_classifier()
```

3. **Train the Model**

Now, you can train the model using the `run_experiment` method. This method requires you to pass the model you've just created, along with your training and testing datasets.

```python
history = vit_model.run_experiment(model, x_train, y_train, x_test, y_test)
```

Make sure you have your dataset (`x_train`, `y_train`, `x_test`, `y_test`) prepared before running this step. The dataset should be pre-processed according to the input requirements of the model, such as resizing images to the expected input shape and normalizing the pixel values.

Here's a complete example in a Jupyter notebook cell:

```python
# Assuming CustomViT class is defined and dataset is loaded and preprocessed
vit_model = CustomViT()

# Create the ViT model
model = vit_model.create_vit_classifier()

# Display the model architecture
model.summary()

# Train and evaluate the model
history = vit_model.run_experiment(model, x_train, y_train, x_test, y_test)
```

This code will initialize the custom Vision Transformer model with the specified parameters, compile it, and train it on your dataset. After training, it will also evaluate the model on your test dataset and print the accuracy and top-5 accuracy metrics.

Remember, the variables `x_train`, `y_train`, `x_test`, and `y_test` need to be defined with your dataset. This example assumes that you have already loaded and pre-processed your data to match the input requirements of the ViT model.

In [ ]:
! pip install tensorflow-addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 13.4 MB/s eta 0:00:00


In [ ]:
from utils import CustomViT as myViT

### `myViT`

In [ ]:
vit_model = myViT(
    num_heads=8,
    image_size=30,
    patch_size=6,
    transformer_layers=8,
    num_epochs=10
)

In [ ]:
model = vit_model.create_vit_classifier()

In [ ]:
tf.test.gpu_device_name()

'/device:GPU:0'

### Training

In [ ]:
%%time

with tf.device('/device:GPU:0'):
    history = vit_model.run_experiment(model, x_train, y_train, x_test, y_test)

Epoch 1/10
3/3 [==============================] - 15s 2s/step - loss: 6.6637 - accuracy: 0.4986 - top-5-accuracy: 1.0000 - val_loss: 1.5524 - val_accuracy: 0.4304 - val_top-5-accuracy: 1.0000
Epoch 2/10
3/3 [==============================] - 1s 228ms/step - loss: 1.2432 - accuracy: 0.4943 - top-5-accuracy: 1.0000 - val_loss: 0.7082 - val_accuracy: 0.4304 - val_top-5-accuracy: 1.0000
Epoch 3/10
3/3 [==============================] - 1s 211ms/step - loss: 0.8969 - accuracy: 0.4786 - top-5-accuracy: 1.0000 - val_loss: 0.7219 - val_accuracy: 0.4304 - val_top-5-accuracy: 1.0000
Epoch 4/10
3/3 [==============================] - 1s 214ms/step - loss: 0.8013 - accuracy: 0.4886 - top-5-accuracy: 1.0000 - val_loss: 0.7253 - val_accuracy: 0.4304 - val_top-5-accuracy: 1.0000
Epoch 5/10
3/3 [==============================] - 1s 215ms/step - loss: 0.7781 - accuracy: 0.4900 - top-5-accuracy: 1.0000 - val_loss: 0.7598 - val_accuracy: 0.4304 - val_top-5-accuracy: 1.0000
Epoch 6/10
3/3 [================

### Tuning

In [ ]:
list_of_val_acc = []
max_val_acc = 0
a, b, c = 4, 2, 2
best_a = 0
list_of_a = []
best_b = 0
list_of_b = []
best_c = 0
list_of_c = []


while curr_max_val_acc < 0.8:
    for a in [4, 8, 12]:
        vit_model = myViT(
            num_heads=a,
            image_size=6*c,
            patch_size=6,
            transformer_layers=b,
            num_epochs=10
        )

        model = vit_model.create_vit_classifier()

        try:
            with tf.device('/device:GPU:0'):
                history = vit_model.run_experiment(model, x_train, y_train, x_test, y_test)
                curr_max_val_acc = np.max(history.history['val_accuracy'])
        except:
            curr_max_val_acc = 0

        list_of_a.append(a)
        list_of_val_acc.append(curr_max_val_acc)
        if curr_max_val_acc > max_val_acc:
            max_val_acc = curr_max_val_acc
        print(f'a: {a}, b: {b}, c: {c}, max_val_acc: {max_val_acc}')

    best_a = max(list_of_a)

    for b in [2, 4, 8, 12]:
        vit_model = myViT(
            num_heads=a,
            image_size=6*c,
            patch_size=6,
            transformer_layers=b,
            num_epochs=10
        )

        model = vit_model.create_vit_classifier()

        try:
            with tf.device('/device:GPU:0'):
                history = vit_model.run_experiment(model, x_train, y_train, x_test, y_test)
                curr_max_val_acc = np.max(history.history['val_accuracy'])
        except:
            curr_max_val_acc = 0

        list_of_b.append(b)
        list_of_val_acc.append(curr_max_val_acc)
        if curr_max_val_acc > max_val_acc:
            max_val_acc = curr_max_val_acc
        print(f'a: {a}, b: {b}, c: {c}, max_val_acc: {max_val_acc}')

    best_b = max(list_of_b)

    for c in [2, 4, 8, 12]:
        vit_model = myViT(
            num_heads=a,
            image_size=6*c,
            patch_size=6,
            transformer_layers=b,
            num_epochs=10
        )

        model = vit_model.create_vit_classifier()

        try:
            with tf.device('/device:GPU:0'):
                history = vit_model.run_experiment(model, x_train, y_train, x_test, y_test)
                curr_max_val_acc = np.max(history.history['val_accuracy'])
        except:
            curr_max_val_acc = 0

        list_of_c.append(c)
        list_of_val_acc.append(curr_max_val_acc)
        if curr_max_val_acc > max_val_acc:
            max_val_acc = curr_max_val_acc
        print(f'a: {a}, b: {b}, c: {c}, max_val_acc: {max_val_acc}')

    best_c = max(list_of_c)

Streaming output truncated to the last 5000 lines.
a: 12, b: 4, c: 12, max_val_acc: 0
a: 12, b: 8, c: 12, max_val_acc: 0
a: 12, b: 12, c: 12, max_val_acc: 0
a: 12, b: 12, c: 2, max_val_acc: 0
a: 12, b: 12, c: 4, max_val_acc: 0
a: 12, b: 12, c: 8, max_val_acc: 0
a: 12, b: 12, c: 12, max_val_acc: 0
a: 4, b: 12, c: 12, max_val_acc: 0
a: 8, b: 12, c: 12, max_val_acc: 0
a: 12, b: 12, c: 12, max_val_acc: 0
a: 12, b: 2, c: 12, max_val_acc: 0
a: 12, b: 4, c: 12, max_val_acc: 0
a: 12, b: 8, c: 12, max_val_acc: 0
a: 12, b: 12, c: 12, max_val_acc: 0
a: 12, b: 12, c: 2, max_val_acc: 0
a: 12, b: 12, c: 4, max_val_acc: 0
a: 12, b: 12, c: 8, max_val_acc: 0
a: 12, b: 12, c: 12, max_val_acc: 0
a: 4, b: 12, c: 12, max_val_acc: 0
a: 8, b: 12, c: 12, max_val_acc: 0
a: 12, b: 12, c: 12, max_val_acc: 0
a: 12, b: 2, c: 12, max_val_acc: 0
a: 12, b: 4, c: 12, max_val_acc: 0
a: 12, b: 8, c: 12, max_val_acc: 0
a: 12, b: 12, c: 12, max_val_acc: 0
a: 12, b: 12, c: 2, max_val_acc: 0
a: 12, b: 12, c: 4, max_val_acc: